# ShorelineMonitor: 40-years of Change Rate

The ShorelineMonitor dataset provides [Satellite-Derived Shorelines (SDS) extracted from annually composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global view of coastal change and shoreline dynamics, serving as a foundation for coastal analytics, modeling and management. The shorelines have been mapped onto the [Global Coastal Transect System](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/gcts/collection.json) to form a [new dataset](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-series/collection.json) of more than 7.5 million time-series. 

This notebook shows how to explore multi-decadal trends in shoreline change that are extracted from the full dataset of time series. The dataset is available upon reasonable request. Please contact the data provider for more information or collaboration opportunities.

In [5]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps
from ipyleaflet import Map as LeafletMap
from ipyleaflet import DrawControl, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [2]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

,type,stac_version,stac_extensions,id,geometry,bbox,links,assets,collection,table:columns,table:row_count,created,proj:code,proj:bbox,datetime,href,alternate_href
0,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n00e135-a9bfdf-133,"POLYGON ((173.95233 0.11202, 173.95233 40.9845...","[134.99191996886077, 0.11202151881534958, 173....",[{'href': 'az://gctr/release/2025-07-08/n00e13...,{'data': {'description': ' A Parquet dataset c...,gctr,[{'description': 'Unique identifier for each t...,77772,2025-07-09 14:10:03.536246+00:00,EPSG:4326,"[134.99191996886077, 0.11202151881534958, 173....",2025-07-08 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,az://gctr/release/2025-07-08/n00e135-a9bfdf-13...
1,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n00e45-0a821d-123,"POLYGON ((90.00195 0.17915, 90.00195 40.9854, ...","[44.995622640381065, 0.1791485568512368, 90.00...",[{'href': 'az://gctr/release/2025-07-08/n00e45...,{'data': {'description': ' A Parquet dataset c...,gctr,[{'description': 'Unique identifier for each t...,351374,2025-07-09 14:10:07.629939+00:00,EPSG:4326,"[44.995622640381065, 0.1791485568512368, 90.00...",2025-07-08 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,az://gctr/release/2025-07-08/n00e45-0a821d-123...
2,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n01w178-a93f11-022,"POLYGON ((-154.79835 0.78747, -154.79835 28.40...","[-178.3130775426825, 0.787472533943265, -154.7...",[{'href': 'az://gctr/release/2025-07-08/n01w17...,{'data': {'description': ' A Parquet dataset c...,gctr,[{'description': 'Unique identifier for each t...,17725,2025-07-09 14:10:20.277934+00:00,EPSG:4326,"[-178.3130775426825, 0.787472533943265, -154.7...",2025-07-08 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,az://gctr/release/2025-07-08/n01w178-a93f11-02...
3,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n04w31-8686b3-033,"POLYGON ((0.01104 4.34594, 0.01104 40.98062, -...","[-31.2777869150827, 4.345937581813707, 0.01103...",[{'href': 'az://gctr/release/2025-07-08/n04w31...,{'data': {'description': ' A Parquet dataset c...,gctr,[{'description': 'Unique identifier for each t...,195622,2025-07-09 14:10:21.256212+00:00,EPSG:4326,"[-31.2777869150827, 4.345937581813707, 0.01103...",2025-07-08 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,az://gctr/release/2025-07-08/n04w31-8686b3-033...
4,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n41e130-dc3ecb-130,"POLYGON ((135.00308 40.97583, 135.00308 43.476...","[129.63771357243888, 40.975831766728795, 135.0...",[{'href': 'az://gctr/release/2025-07-08/n41e13...,{'data': {'description': ' A Parquet dataset c...,gctr,[{'description': 'Unique identifier for each t...,15859,2025-07-09 14:10:28.078116+00:00,EPSG:4326,"[129.63771357243888, 40.975831766728795, 135.0...",2025-07-08 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,az://gctr/release/2025-07-08/n41e130-dc3ecb-13...


In [6]:
from coastpy.geo.utils import get_region_of_interest_from_map
from ipyleaflet.leaflet import DrawControl

# Crate a new map with ipyleaflet
draw_aoi = True
if draw_aoi:
    m  = LeafletMap(basemap=basemaps.OpenStreetMap.Mapnik, zoom=3, scroll_wheel_zoom=True, layout={'height': '600px'})
    draw_control = DrawControl()
    m.add_control(draw_control)
    display(m)
    


Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [7]:
if draw_aoi:
    # Draw area of interest
    name = 'Draw'
    bounds = gpd.GeoDataFrame.from_features(draw_control.data, crs="EPSG:4326").total_bounds
else:
    # Select area of interest
    aoi_idx = 1
    aois = [{'name': 'Saint Louis Senegal', 'bounds': (-16.5675862529, 15.7355463165, -16.454518612, 16.0670520835)}]
    name = aois[aoi_idx]['name']
    bounds = aois[aoi_idx]['bounds']

print(f"Selected AOI: {name} ({bounds})")
roi = gpd.GeoDataFrame(geometry=[shapely.geometry.box(*bounds)], crs="EPSG:4326")
west, south, east, north = list(roi.total_bounds)

Selected AOI: Draw ([-9.997229 31.341643 -8.917774 32.20789 ])


## Fetch data from the database

In [8]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    columns = ["transect_id", "lon", "lat", "geometry", "continent", "sds:change_rate", "sds:tr_stdev", "class:shore_type"]  # when you don't need all data
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
df.head()

,transect_id,lon,lat,geometry,continent,sds:change_rate,sds:tr_stdev,class:shore_type
0,cl30402s01tr00559358,-9.346251,32.033108,"LINESTRING (-9.33684 32.02898, -9.35567 32.03724)",AF,2.277674,67.611732,sandy_gravel_or_small_boulder_sediments
1,cl30402s01tr00559258,-9.345766,32.033909,"LINESTRING (-9.33635 32.02978, -9.35518 32.03804)",AF,2.088813,78.146248,sandy_gravel_or_small_boulder_sediments
2,cl30402s01tr00559158,-9.345281,32.034710,"LINESTRING (-9.33587 32.03058, -9.3547 32.03884)",AF,0.817576,39.692093,sandy_gravel_or_small_boulder_sediments
3,cl30402s01tr00559058,-9.344796,32.035511,"LINESTRING (-9.33538 32.03138, -9.35421 32.03964)",AF,0.674791,33.146034,sandy_gravel_or_small_boulder_sediments
4,cl30402s01tr00558958,-9.344344,32.036327,"LINESTRING (-9.33435 32.03335, -9.35434 32.03931)",AF,0.438558,27.773962,sandy_gravel_or_small_boulder_sediments
